### Парсинг данных с помощью Beautiful Soup
__План работ:__
<ol>
    <li>Импорт библиотек</li>
    <li>Список адресов для парсинга</li>
    <li>Парсинг и пополнение списков</li>
    <li>Сохранение в csv</li>
</ol>

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
# Задаем базовую ссылку
url = "https://habr.com/ru"
first_page = 692500
last_page = 692510

# Функция генерации адресов
def get_urls():
    list_url = [];
    for uID in range(first_page, last_page):
        page_url = "{}/post/{}".format(url, uID)
        list_url.append(page_url)
    return list_url

print(f"Общее кол-во сгенерированных URL: {len(get_urls())}")

Общее кол-во сгенерированных URL: 10


In [3]:
titl_list = [] # Заголовки
auth_list = [] # Авторы
urls_list = [] # OK URL

In [4]:
def get_data():
    urls_list.clear()
    for at_url in get_urls():
        at_data = requests.get(at_url)
        if at_data.status_code == 200:
            urls_list.append(at_url)
            print(f"{at_url} - OK")

get_data()
print(f"Успешно получено {len(urls_list)} из {len(get_urls())} URL")

https://habr.com/ru/post/692500 - OK
https://habr.com/ru/post/692502 - OK
https://habr.com/ru/post/692504 - OK
https://habr.com/ru/post/692506 - OK
Успешно получено 4 из 10 URL


In [5]:
def parse_data(urls_list):    
    for at_url in urls_list:
        at_data = requests.get(at_url)
        if at_data.status_code == 200:            
            soup_data = BeautifulSoup(at_data.text, "html5lib")            
            # Парсим заголовки
            titles = soup_data.findAll("h1", class_ = "tm-article-snippet__title tm-article-snippet__title_h1")
            for title_ind in range(len(titles)):
                if titles[title_ind].find("span") is not None:
                    titl_list.append(titles[title_ind].text)
                    print(f"{at_url} - {titles[title_ind].text.strip()}")
                else:
                    print(f"{at_url} - Заголовок не найден")
            # Парсим авторов
            authors = soup_data.findAll("span", class_ = "tm-user-info__user")
            for author_ind in range(len(authors)):
                if authors[author_ind].find("a", class_ = "tm-user-info__username") is not None:
                    auth_list.append(authors[author_ind].text.strip())
            
parse_data(urls_list)

https://habr.com/ru/post/692500 - От канадского телекома до WikiLeaks: путь Ассанжа
https://habr.com/ru/post/692502 - Голосовой ассистент «Алиса» научилась узнавать всех членов семьи
https://habr.com/ru/post/692504 - Создаем Telegram бот с регистрацией пользователей из таблицы в Airtable + Python
https://habr.com/ru/post/692506 - Знакомство вслепую: Bumble заимствовал идею у «Теда Лассо»


In [6]:
print ("Список авторов:")
for author in auth_list:
    print(f"\"{author}\"")

Список авторов:
"Travis_Macrif"
"denis-19"
"TAU15"
"Markaty"


In [7]:
# Создаем датаферйм из данных
data_dict = {"author": auth_list, "title": titl_list, "url": urls_list}
df = pd.DataFrame.from_dict(data_dict, orient = "index")
df = df.transpose() # Индексы должны оказаться вверху
df.head()

,author,title,url
0,Travis_Macrif,От канадского телекома до WikiLeaks: путь Ассанжа,https://habr.com/ru/post/692500
1,denis-19,Голосовой ассистент «Алиса» научилась узнавать...,https://habr.com/ru/post/692502
2,TAU15,Создаем Telegram бот с регистрацией пользовате...,https://habr.com/ru/post/692504
3,Markaty,Знакомство вслепую: Bumble заимствовал идею у ...,https://habr.com/ru/post/692506


In [8]:
# Сохраняем в файл
df.to_csv("habr.csv", sep = ",", index = False)